In [266]:
import pandas as pd
pd.set_option('display.max_columns', None)  
# df = pd.read_csv(r"D:\EDrivebackup\brocamp\techTest\ecommerce-clone-and-review-insights\shoppify_review_analysis\data\sample_customer_reviews_raw - sample_customer_reviews_raw.csv.csv")
df=pd.read_csv('D:\EDrivebackup/brocamp/techTest/ecommerce-clone-and-review-insights/shoppify_review_analysis/data/customerdata.csv')
df.head()


,Review ID,Product Name,Rating,Review Content,Timestamp,Customer Email,Product Category,Order Value,Fulfillment Status,Shipping Country
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Cross-group web-enabled encoding,NaN,Pressure let kind degree list top somebody col...,2025-03-04 1:23:13,williamsdarlene@baker.org,Evening Gowns,NaN,Fulfilled,Germany
1,23b8c1e9-3924-46de-beb1-3b9046685257,Down-sized incremental matrix,1.0,Challenge reach throughout team those sing des...,2025-05-10 16:21:32,ismall@may-turner.com,Prom Dresses,197.72,NaN,Australia
2,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,Polarized didactic initiative,1.0,Lead soon property.,2025-03-09 11:07:09,shawn02@yahoo.com,Cocktail Dreses,485.10,NaN,Germany
3,972a8469-1641-4f82-8b9d-2434e465e150,Centralized composite implementation,NaN,Other wear through partner rest.,2025-05-18 18:41:54,vmarshall@hotmail.com,PromDrsses,338.67,Delayed,United States
4,17fc695a-07a0-4a6e-8822-e8f36c031199,Adaptive human-resource firmware,3.0,Interview trade knowledge city technology late...,2025-01-13 14:38:16,michelle55@anderson-wood.net,Plus Size Dresses,491.52,Cancelled,Germany


In [267]:
df.dtypes

Review ID              object
Product Name           object
Rating                float64
Review Content         object
Timestamp              object
Customer Email         object
Product Category       object
Order Value           float64
Fulfillment Status     object
Shipping Country       object
dtype: object

In [268]:
df.describe()

,Rating,Order Value
count,80.000000,97.000000
mean,2.762500,275.804742
std,1.425303,139.536561
min,1.000000,50.180000
25%,1.750000,148.740000
50%,3.000000,296.540000
75%,4.000000,386.380000
max,5.000000,499.960000


In [269]:
df['Customer Email'].value_counts()

williamsdarlene@baker.org    1
comptonshelly@hotmail.com    1
michael77@gmail.com          1
powens@yahoo.com             1
srocha@padilla.com           1
                            ..
sandra47@harris.com          1
kellyroberts@hotmail.com     1
david86@yahoo.com            1
ybailey@hotmail.com          1
sean41@miller.info           1
Name: Customer Email, Length: 100, dtype: int64

In [270]:
import re

# Define email pattern
email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'

# Filter rows with valid emails
df = df[df['Customer Email'].str.match(email_pattern, na=False)]



In [271]:
df.shape

(100, 10)

In [303]:
df['Shipping Country'].unique()

array(['Germany', 'Australia', 'United States', 'Canada',
       'United Kingdom'], dtype=object)

In [272]:
df['Fulfillment Status'].value_counts()

Cancelled    17
Returned     16
Fufilled     14
Delaye       13
Fulfilled    12
Delayed       9
Name: Fulfillment Status, dtype: int64

In [273]:
df['Fulfillment Status'] = df['Fulfillment Status'].str.strip().str.lower()
status_corrections = {
    'delaye': 'delayed',
    'fufilled': 'fulfilled'
}
df['Fulfillment Status'] = df['Fulfillment Status'].replace(status_corrections)

In [274]:
df['Fulfillment Status'].value_counts()

fulfilled    26
delayed      22
cancelled    17
returned     16
Name: Fulfillment Status, dtype: int64

In [275]:
df['Fulfillment Status'].unique()

array(['fulfilled', nan, 'delayed', 'cancelled', 'returned'], dtype=object)

In [276]:
df['Fulfillment Status'] = df['Fulfillment Status'].fillna(df['Fulfillment Status'].mode()[0])

In [277]:
df.isna().sum()

Review ID              0
Product Name           0
Rating                20
Review Content         8
Timestamp              0
Customer Email         0
Product Category       7
Order Value            3
Fulfillment Status     0
Shipping Country       0
dtype: int64

In [278]:
# Standardize timestamps
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")


In [279]:
df["Review Content"][:2]

0    Pressure let kind degree list top somebody col...
1    Challenge reach throughout team those sing des...
Name: Review Content, dtype: object

In [280]:
# Filter out blank/invalid reviews
df = df[df["Review Content"].str.len() > 10]


In [281]:

missing_ratings = df[df["Rating"].isnull()]
print(f"Missing ratings: {len(missing_ratings)}")

Missing ratings: 19


In [282]:
df['Rating'].unique()



array([nan,  1.,  3.,  2.,  5.,  4.])

In [283]:
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")


In [284]:
df['Rating'].unique()

array([nan,  1.,  3.,  2.,  5.,  4.])

In [285]:
# Normalize missing fields
df["Rating_Missing"] = df["Rating"].isna().astype(int)

df["Rating"] = df.groupby("Product Category")["Rating"].transform(lambda x: x.fillna(x.median()))
df["Rating"] = df["Rating"].fillna(df["Rating"].median())
df['Order Value']=df['Order Value'].fillna(df['Order Value'].mean())

In [286]:
df.isna().sum()

Review ID             0
Product Name          0
Rating                0
Review Content        0
Timestamp             0
Customer Email        0
Product Category      6
Order Value           0
Fulfillment Status    0
Shipping Country      0
Rating_Missing        0
dtype: int64

In [287]:
# Fix inconsistent category/product names
df["Product Category"] = df["Product Category"].str.strip().str.lower()
df["Product Name"] = df["Product Name"].str.strip().str.lower()

In [288]:
dress_corrections = {
    'promdrsses': 'prom dresses',
    'cocktail dreses': 'cocktail dresses'
}
df['Product Category'] = df['Product Category'].replace(dress_corrections)


In [289]:
df['Product Category'].value_counts()

prom dresses          25
cocktail dresses      21
plus size dresses     16
bridesmaid dresses    14
evening gowns         10
Name: Product Category, dtype: int64

In [290]:
df.isna().sum()

Review ID             0
Product Name          0
Rating                0
Review Content        0
Timestamp             0
Customer Email        0
Product Category      6
Order Value           0
Fulfillment Status    0
Shipping Country      0
Rating_Missing        0
dtype: int64

In [291]:
df['Product Category'].unique()

array(['evening gowns', 'prom dresses', 'cocktail dresses',
       'plus size dresses', 'bridesmaid dresses', nan], dtype=object)

In [292]:
df['Product Category'].value_counts()

prom dresses          25
cocktail dresses      21
plus size dresses     16
bridesmaid dresses    14
evening gowns         10
Name: Product Category, dtype: int64

In [293]:
df['Product Category']=df['Product Category'].fillna(df['Product Category'].mode()[0])

In [294]:
df.to_csv(r"D:\EDrivebackup\brocamp\techTest\ecommerce-clone-and-review-insights\shoppify_review_analysis\data\cleaned.csv", index=False)

In [295]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"), model="llama3-8b-8192")


In [ ]:
from langchain.tools import Tool
from pandasai import SmartDataframe


import pandas as pd


smart_df = SmartDataframe(df, config={"llm": llm, "verbose": True})

pandasai_tool = Tool(
    name="Dataframe Insight Tool",
    func=lambda q: smart_df.chat(q),
    description="Useful for asking questions about the uploaded dataframe"
)

response = pandasai_tool.run("What is the average rating by product category?")
print(response)


     Product Category    Rating
0  bridesmaid dresses  4.428571
1    cocktail dresses  2.714286
2       evening gowns  3.400000
3   plus size dresses  2.875000
4        prom dresses  2.387097


In [297]:
response = pandasai_tool.run("Which product categories have the most 1-star reviews in Canada?")
print(response)


  Product Category  Count
0     prom dresses      2


In [298]:
response = pandasai_tool.run("Do higher order values correlate with lower ratings?")
print(response)


{'type': 'string', 'value': 'The correlation between Order Value and Rating is -0.04. A negative correlation suggests higher order values may correlate with lower ratings.'}
The correlation between Order Value and Rating is -0.04. A negative correlation suggests higher order values may correlate with lower ratings.


In [306]:

response = pandasai_tool.run("Can you summarize the top 5 complaints and 5 compliments across all reviews?")
print(response)

{'type': 'string', 'value': "Top 5 Complaints: ['Pressure let kind degree list top somebody college be middle plan friend stand seem pull different current agency each little sure authority increase picture create recent manager.', 'Ago upon stop environment Congress reflect finally fund accept thank not.', 'Option goal avoid left also hard expert popular.', 'Ten physical character attention kind field.', 'Less nor often race think poor worry.']\nTop 5 Compliments: ['Pressure let kind degree list top somebody college be middle plan friend stand seem pull different current agency each little sure authority increase picture create recent manager.', 'Ago upon stop environment Congress reflect finally fund accept thank not.', 'Option goal avoid left also hard expert popular.', 'Ten physical character attention kind field.', 'Less nor often race think poor worry.']"}
Top 5 Complaints: ['Pressure let kind degree list top somebody college be middle plan friend stand seem pull different curren

In [ ]:
response = pandasai_tool.run("Which fulfillment statuses are most associated with negative feedback?")
print(response)

{'type': 'dataframe', 'value':   Fulfillment Status    Rating
0          cancelled  2.500000
2          fulfilled  2.111111
1            delayed  2.076923}
  Fulfillment Status    Rating
0          cancelled  2.500000
2          fulfilled  2.111111
1            delayed  2.076923
